In [3]:
import pandas as pd
import numpy as np
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets

from google.colab import drive
drive.mount('/content/gdrive')

train_normal= np.load('/content/gdrive/MyDrive/Data/train_Normal_128.npy')
train_virus= np.load('/content/gdrive/MyDrive/Data/train_Virus_128.npy')
test_normal= np.load('/content/gdrive/MyDrive/Data/test_Normal_128.npy')
test_virus= np.load('/content/gdrive/MyDrive/Data/test_Virus_128.npy')

ttn= np.float32(test_normal)
torch.manual_seed(3)

tensor_train_normal = torch.Tensor(train_normal)
tensor_train_virus = torch.Tensor(train_virus)
tensor_test_normal = torch.Tensor(ttn)
tensor_test_virus = torch.Tensor(test_virus)

Train = torch.cat((tensor_train_normal,tensor_train_virus))
Test = torch.cat((tensor_test_normal,tensor_test_virus))

Shuffle_Train= Train[torch.randperm(Train.size()[0])]
Shuffle_Test= Test[torch.randperm(Test.size()[0])]

Final_Train= Shuffle_Train[3:int(torch.rand(1,)*1000), :, :,:] 
Final_Test= Shuffle_Test[3:int(torch.rand(1,)*1000), :, :,:]

Size_Train=len(Final_Train)
Size_Test=len(Final_Test)
print(Size_Train)
print(Size_Test)
train_y = torch.randint(0,9, (Size_Train,))   
test_y = torch.randint(0,9, (Size_Test,))



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
36
379


In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.cnn_layer_1 = nn.Conv2d(in_channels=1, out_channels=16,kernel_size=5, stride=1, padding=2)
        self.cnn_layer_2 = nn.Conv2d(in_channels=16, out_channels=32,kernel_size=5, stride=1, padding=2)
        
        self.flatten = nn.Flatten()
        self.maxpool = nn.MaxPool2d(2,2)
        
        self.linear_layer_1 = nn.Linear(32768, 512) 
        self.linear_layer_2 = nn.Linear(512, 128)
        self.linear_layer_3 = nn.Linear(128, 10)   
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(.2)
        
       
        
    def forward(self, x):

        x = self.cnn_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
       
        
        x = self.cnn_layer_2(x)   #print(x.shape)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        #print(x.shape)
        
        x = self.flatten(x)
        #print(x.shape)
        
        x = self.linear_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        x = self.linear_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        x = self.linear_layer_3(x)
       
        return x

In [5]:
model = NeuralNetwork(16384)
print(model)



NeuralNetwork(
  (cnn_layer_1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cnn_layer_2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_layer_1): Linear(in_features=32768, out_features=512, bias=True)
  (linear_layer_2): Linear(in_features=512, out_features=128, bias=True)
  (linear_layer_3): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.2, inplace=False)
)


In [6]:
from torchsummary import summary
summary(model,(1,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 128, 128]             416
           Dropout-2         [-1, 16, 128, 128]               0
              ReLU-3         [-1, 16, 128, 128]               0
         MaxPool2d-4           [-1, 16, 64, 64]               0
            Conv2d-5           [-1, 32, 64, 64]          12,832
           Dropout-6           [-1, 32, 64, 64]               0
              ReLU-7           [-1, 32, 64, 64]               0
         MaxPool2d-8           [-1, 32, 32, 32]               0
           Flatten-9                [-1, 32768]               0
           Linear-10                  [-1, 512]      16,777,728
          Dropout-11                  [-1, 512]               0
             ReLU-12                  [-1, 512]               0
           Linear-13                  [-1, 128]          65,664
          Dropout-14                  [

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.001)

In [8]:
def trainModel(model, loss_fn, optimizer):
    model.train()

    batch = 100

    loss = 0
    for i in range(Final_Train.shape[0]):
      x, y = torch.reshape(Final_Train[i],(1,1,128,128)), torch.tensor([train_y[i]], dtype=torch.float)
      label=torch.zeros([1,10,], dtype=torch.float32)
      label[0,int(y.item())]=1
      # Compute prediction error
      pred = model(x)     
      #print(pred)
      #print(label)
      loss += loss_fn(pred, label)
      
      if i>0 and (i+1)%batch == 0:
          # Backpropagation
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          print(f'Training Loss: {loss.item():.4f}', end="\r")
          loss = 0
    print()

In [9]:
def testModel(model, loss_fn):
    model.eval()

    size = Final_Test.shape[0]
    correct=0
    loss = 0
    total =10000
    with torch.no_grad():
      for i in range(Final_Test.shape[0]):
        x, y = torch.reshape(Final_Test[i],(1,1,128,128)), torch.tensor([test_y[i]], dtype=torch.float)
        label=torch.zeros([1,10,], dtype=torch.float32)
        label[0,int(y.item())]=1
        pred = model(x)
        #print(pred)
        predicted = torch.argmax(pred)
        #print(predicted)
        #print(y)


        # Total correct predictions
        correct += (predicted == int(y)).sum()

        loss += loss_fn(pred, label)
      
    loss /= size
    accuracy = 100 * correct.item() / total

    #Print Loss
    print('Loss: {}. Accuracy: {}'.format({loss}, accuracy))
    

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    trainModel(model, loss_fn, optimizer)
    testModel(model, loss_fn)
print("Done!")

Epoch 1
-------------------------------

Loss: {tensor(2.6122)}. Accuracy: 0.44
Epoch 2
-------------------------------

Loss: {tensor(2.6122)}. Accuracy: 0.44
Epoch 3
-------------------------------

Loss: {tensor(2.6122)}. Accuracy: 0.44
Epoch 4
-------------------------------

Loss: {tensor(2.6122)}. Accuracy: 0.44
Epoch 5
-------------------------------

Loss: {tensor(2.6122)}. Accuracy: 0.44
Done!
